# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
import pyspark
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 
from pyspark.sql.functions import udf, desc
import pyspark.sql.functions as func
from pyspark.sql.window import Window

from pyspark.sql.types import *
from pyspark.sql.types import FloatType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
import pandas as pd 

import datetime

ss = pyspark.sql.SparkSession.builder.getOrCreate()

df_ss_json = ss.read.json("data/sparkify_log_small.json")
df_ss_json.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

# Question 1

Which page did user id "" (empty string) NOT visit?

In [2]:
# TODO: write your code to answer question 1
df_pd_json = df_ss_json.select("page").dropDuplicates().toPandas()
#df_pd_json.head(5)
list_pages_full_pd = df_pd_json["page"].tolist()
print("All pages:", list_pages_full_pd)

df_pd_json_userid = df_ss_json.filter(df_ss_json["userId"]=="").select("page").dropDuplicates().toPandas()
list_pages_userid_pd = df_pd_json_userid["page"].tolist()
print("pages visited by user id \"\" : ", list_pages_userid_pd)

list_diff = list()
for item in list_pages_full_pd:
    if item not in list_pages_userid_pd:
        list_diff.append(item)
print("pages not visited by user id \"\" : ", list_diff)

All pages: ['Submit Downgrade', 'Home', 'Downgrade', 'Logout', 'Save Settings', 'About', 'Settings', 'Login', 'NextSong', 'Help', 'Upgrade', 'Error', 'Submit Upgrade']
pages visited by user id "" :  ['Home', 'About', 'Login', 'Help']
pages not visited by user id "" :  ['Submit Downgrade', 'Downgrade', 'Logout', 'Save Settings', 'Settings', 'NextSong', 'Upgrade', 'Error', 'Submit Upgrade']


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [3]:
# TODO: use this space to explore the behavior of the user with an empty string


# Question 3

How many female users do we have in the data set?

In [3]:
# TODO: write your code to answer question 3
female_count = df_ss_json.filter(df_ss_json["gender"].isin(["F"])).select("userID").dropDuplicates().count()
print("female count: ", female_count)
df_gender_count = df_ss_json.select("userID", "gender").dropDuplicates().groupBy("gender").count()
print(df_gender_count.show())

female count:  462
+------+-----+
|gender|count|
+------+-----+
|     F|  462|
|  null|    1|
|     M|  501|
+------+-----+

None


# Question 4

How many songs were played from the most played artist?

In [4]:
# TODO: write your code to answer question 4
df_ss_json.filter(df_ss_json.page == "NextSong").select("artist").\
    groupBy("artist").count().sort(desc("count")).show(10)

#df_ss_json.filter(df_ss_json.page == "NextSong").select("artist").\
#    groupBy("artist").count().sort(desc("count")).collect()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|        Jack Johnson|   36|
|                Muse|   36|
+--------------------+-----+
only showing top 10 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [35]:
# TODO: write your code to answer question 5
func_is_home = udf(lambda x: int(x=="Home"), IntegerType())

df_ss_json_f = df_ss_json.filter((df_ss_json.page=="Home")| (df_ss_json.page=="NextSong"))
#print(df_ss_json_f.count())

df_ss_json_f_1 = df_ss_json_f.withColumn("Is_Home", func_is_home("page"))
#df_ss_json_f_1.show(5)

df_ss_json_f_2 = df_ss_json_f_1.select("userId", "ts", "page", "Is_Home") 
#df_ss_json_f_2.show(5)

window_def = Window.partitionBy("userId").orderBy("ts").\
    rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df_ss_json_f_3 = df_ss_json_f_2.withColumn("total_count", func.count("ts").over(window_def))
#df_ss_json_f_3.show(50)
df_ss_json_f_4 = df_ss_json_f_3.withColumn("home_count", func.sum("Is_Home").over(window_def))
df_ss_json_f_5 = df_ss_json_f_4.withColumn("nextsong_count", (df_ss_json_f_4.total_count - df_ss_json_f_4.home_count))
#df_ss_json_f_5.show(50)
df_ss_json_f_6 = df_ss_json_f_5.select("userId", "home_count", "nextsong_count").dropDuplicates()
df_ss_json_f_6.show(50)
print(df_ss_json_f_6.groupBy().sum("home_count").collect()[0][0])
print(df_ss_json_f_6.groupBy().sum("nextsong_count").collect()[0][0])
print("Average nextsong_count per home_count = ", df_ss_json_f_6.groupBy().sum("nextsong_count").collect()[0][0] \
     / df_ss_json_f_6.groupBy().sum("home_count").collect()[0][0])


+------+----------+--------------+
|userId|home_count|nextsong_count|
+------+----------+--------------+
|  1436|         0|             2|
|  2088|         1|            13|
|  2162|         2|            34|
|  2294|         5|            55|
|  2904|         0|             1|
|   691|         1|             3|
|  1394|         1|            26|
|  2275|         1|             3|
|  2756|         2|             5|
|   451|         2|             2|
|   800|         1|             2|
|  1280|         2|             0|
|  1746|         2|             0|
|  2696|         1|             1|
|   870|         0|             2|
|   926|         3|             4|
|  1903|         2|             5|
|   591|         0|             1|
|     7|         0|             2|
|   307|         0|             1|
|   574|         2|             4|
|   613|         1|             2|
|  1043|         1|             1|
|  2402|         1|             3|
|  2644|         0|             1|
|   577|         1| 